In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
os.environ["QUANTIZE_LLAMA_31"] = "1"

In [2]:
import torch
print(torch.__version__)
import torch._custom_ops

2.1.0+cu121


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from quantizer import QuipQuantizer
import os

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.device_count()

2

# Llama 2 bit

In [6]:
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_name = "VityaVitalich/Llama3.1-8b-instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# quant_dir = "/root/llm_quant_safety/quantization/models/Mistral-7B-Instruct-v0.2-quip-2bit"
quant_dir = "/root/llm_quant_safety/quantization/models/Llama-3.1-8B-Instruct-quip-2bit"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb): L

In [9]:
import logging
def setup_logging():
    # Настройка корневого логгера
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(),  # терминал
            logging.FileHandler('logs/llama_e8p12.log')
        ]
    )

In [10]:
setup_logging()

In [11]:
quant = QuipQuantizer(
    # codebook="E8P12RVQ4B", # 4 bit
    codebook="E8P12", # 2 bit
    dataset="wikitext2",
    nsamples=1024, # 4096 - default ~500-750 CPU mem
    ft_train_size=256,
    # cache_on_gpu=True,
    # model_seqlen=131072,
    model_seqlen=1024
)

In [12]:
quant.quantize_model(model, tokenizer, quant_dir)

Token indices sequence length is longer than the specified maximum sequence length for this model (2436214 > 131072). Running this sequence through the model will result in indexing errors


Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

2024-09-07 06:37:11,351 - quantizer - INFO - Start quantizing block model.layers 1/32
2024-09-07 06:37:11,355 - quantizer - INFO - Module to quantize ['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.o_proj', 'mlp.gate_proj', 'mlp.up_proj', 'mlp.down_proj']


Quantizing layers inside the block: 0it [00:00, ?it/s]

2024-09-07 06:59:48,238 - quantizer - INFO - Quantizing self_attn.q_proj in block 1/32...
2024-09-07 07:00:36,404 - quantizer - INFO - Quantizing self_attn.k_proj in block 1/32...
2024-09-07 07:00:51,467 - quantizer - INFO - Quantizing self_attn.v_proj in block 1/32...
2024-09-07 07:01:11,991 - quantizer - INFO - Block 1 initial loss 4.050996780395508
2024-09-07 07:01:55,691 - quantizer - INFO - Quantizing self_attn.o_proj in block 1/32...
2024-09-07 07:02:47,539 - quantizer - INFO - Block 1 initial loss 61146.3984375
2024-09-07 07:03:29,298 - quantizer - INFO - Quantizing mlp.gate_proj in block 1/32...
2024-09-07 07:06:05,750 - quantizer - INFO - Quantizing mlp.up_proj in block 1/32...
2024-09-07 07:08:44,941 - quantizer - INFO - Block 1 initial loss 77009.609375
2024-09-07 07:09:20,240 - quantizer - INFO - Quantizing mlp.down_proj in block 1/32...
2024-09-07 07:12:04,174 - quantizer - INFO - Start quantizing block model.layers 2/32
2024-09-07 07:12:04,177 - quantizer - INFO - Module 

...

Quantizing layers inside the block: 0it [00:00, ?it/s]

2024-09-08 00:45:18,206 - quantizer - INFO - Quantizing self_attn.q_proj in block 31/32...
2024-09-08 00:46:04,653 - quantizer - INFO - Quantizing self_attn.k_proj in block 31/32...
2024-09-08 00:46:19,452 - quantizer - INFO - Quantizing self_attn.v_proj in block 31/32...
2024-09-08 00:46:39,424 - quantizer - INFO - Block 31 initial loss 110.40959930419922
2024-09-08 00:46:56,003 - quantizer - INFO - Block 31 @ epoch 0 new loss 101.96656036376953 old loss 110.40959930419922 BETTER
2024-09-08 00:47:10,354 - quantizer - INFO - Block 31 @ epoch 1 new loss 101.14036560058594 old loss 101.96656036376953 BETTER
2024-09-08 00:47:24,276 - quantizer - INFO - Block 31 @ epoch 2 new loss 100.34866333007812 old loss 101.14036560058594 BETTER
2024-09-08 00:47:38,719 - quantizer - INFO - Block 31 @ epoch 3 new loss 98.1333999633789 old loss 100.34866333007812 BETTER
2024-09-08 00:47:52,661 - quantizer - INFO - Block 31 @ epoch 4 new loss 97.24917602539062 old loss 98.1333999633789 BETTER
2024-09-08 

Quantizing layers inside the block: 0it [00:00, ?it/s]

2024-09-08 01:21:05,006 - quantizer - INFO - Quantizing self_attn.q_proj in block 32/32...
2024-09-08 01:21:51,718 - quantizer - INFO - Quantizing self_attn.k_proj in block 32/32...
2024-09-08 01:22:05,950 - quantizer - INFO - Quantizing self_attn.v_proj in block 32/32...
2024-09-08 01:22:26,216 - quantizer - INFO - Block 32 initial loss 357.33258056640625
2024-09-08 01:22:41,990 - quantizer - INFO - Block 32 @ epoch 0 new loss 353.2872314453125 old loss 357.33258056640625 BETTER
2024-09-08 01:23:21,745 - quantizer - INFO - Block 32 @ epoch 3 new loss 351.0727844238281 old loss 353.2872314453125 BETTER
2024-09-08 01:23:35,326 - quantizer - INFO - Quantizing self_attn.o_proj in block 32/32...
2024-09-08 01:24:27,379 - quantizer - INFO - Block 32 initial loss 1647864.25
2024-09-08 01:25:08,925 - quantizer - INFO - Quantizing mlp.gate_proj in block 32/32...
2024-09-08 01:27:43,235 - quantizer - INFO - Quantizing mlp.up_proj in block 32/32...
2024-09-08 01:30:22,550 - quantizer - INFO - Bl

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (q_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (v_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (o_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
        )
        (mlp): LlamaMLP(
          (act_fn): SiLU()
          (gate_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (up_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (down_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMS

In [13]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (q_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (v_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (o_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
        )
        (mlp): LlamaMLP(
          (act_fn): SiLU()
          (gate_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (up_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
          (down_proj): QuantLinear(
            (codebook): E8P12_codebook()
          )
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMS

In [14]:
quant.save(model, quant_dir)

2024-09-04 06:04:55,250 - accelerate.accelerator - INFO - Model weights saved in /root/llm_quant_safety/quantization/models/Mistral-7B-Instruct-v0.2-quip-2bit/pytorch_model.bin


### try inference

In [16]:
# tokenizer.padding_side="left"
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# input_ids = tokenizer.encode("The capital of France is", return_tensors="pt").cuda()
# print(tokenizer.decode(model.generate(input_ids, do_sample=True, temperature=1)[0]))

# TMP quantizer exploration

In [19]:
from utils import get_block_name_with_pattern, get_preceding_modules, recurse_getattr

In [13]:
get_block_name_with_pattern(model)

'model.layers'

In [15]:
model.model.layers

ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
      (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
      (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
      (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)

In [17]:
get_preceding_modules(model, 'model.layers')

['model.embed_tokens']

In [20]:
blocks = recurse_getattr(model, 'model.layers')

In [21]:
blocks

ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
      (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
      (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
      (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)

In [23]:
module_names_after_last_block = get_preceding_modules(model, 'model.layers', reverse=True)

In [24]:
module_names_after_last_block

['lm_head', 'model.rotary_emb', 'model.norm']